In [ ]:
%load_ext autoreload
%autoreload 2

# Matplotlib DEBUG logging spits out a whole bunch of crap
import logging
import matplotlib
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)
font = {'family': 'normal',
        'weight': 'bold',
        'size': 12}
matplotlib.rc('font', **font)
from context import pid_controller

In [ ]:
from copy import deepcopy

import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Layout

from pid_controller.controller import LinearSystem, Sensor, ProportionalController, ProportionalDifferentialController, MassSystem, PidController
from pid_controller.loop import closed_loop
from pid_controller.plotting import setup_plt_figure
from pid_controller.plotting import plot_control_loop_output

In [ ]:
layout = Layout(width='50%', description_width='initial')
style = {'description_width': '30%'}

widget_sliders = {
    'init_state':         widgets.FloatSlider(value=1.0, min=-10, max=10, step=0.1, description='Initial Position', layout=layout, style=style),
    'desired_state':      widgets.FloatSlider(value=5.0, min=-10, max=10, step=0.1, description='Desired Position', layout=layout, style=style),
    'system_noise_std':   widgets.FloatSlider(value=0.01, min=0.0, max=0.05, step=0.01, readout_format='.3f', description='System Noise', layout=layout, style=style),
    'sensor_noise_std':   widgets.FloatSlider(value=0.01, min=0.0, max=0.05, step=0.01, description='Sensor Noise', layout=layout, style=style),
    'p_gain':             widgets.FloatSlider(value=0.01, min=-1.0, max=3.0, step=0.05, description='Gain', layout=layout, style=style),
    'eps':                widgets.FloatSlider(value=0.001, min=0.0001, max=0.1, step=0.001, description='Setpoint Accuracy', layout=layout, style=style),
    'time_delta':         widgets.FloatSlider(value=1.0, min=0.001, max=5.0, step=0.01, description='Time Step', layout=layout, style=style),
    'max_steps':          widgets.FloatSlider(value=200, min=1, max=1000, step=10, description='Max Steps', layout=layout, style=style),
    'max_time':           widgets.FloatSlider(value=200, min=1, max=1000, step=10, description='Max Time', layout=layout, style=style)
}

@interact(**widget_sliders)
def run_p_control(init_state, system_noise_std, sensor_noise_std, p_gain, desired_state, eps, time_delta, max_steps, max_time):
    linear_system = LinearSystem(init_state, system_noise_std)
    sensor = Sensor(noise_std=sensor_noise_std)
    p_controller = ProportionalController(p_gain)

    output_generator = closed_loop(linear_system, p_controller, sensor, desired_state=desired_state, init_velocity=0.0,
                                   eps=eps, time_delta=time_delta, max_steps=max_steps, max_time=max_time)

    fig = plot_control_loop_output(output_generator, p_controller)

In [ ]:
PRINT_DEBUG       = False

layout = Layout(width='50%', description_width='initial')
style = {'description_width': '30%'}

widget_sliders = {
    'init_state':         widgets.FloatSlider(value=1.0,   min=-10,    max=10,   step=0.1,   description='Initial Position', layout=layout, style=style),
    'init_velocity':      widgets.FloatSlider(value=0.0,   min=-10,    max=10,   step=0.1,   description='Initial Velocity', layout=layout, style=style),
    'desired_state':      widgets.FloatSlider(value=5.0,   min=-10,    max=10,   step=0.1,   description='Desired Position', layout=layout, style=style),
    'desired_velocity':   widgets.FloatSlider(value=5.0,   min=-10,    max=10,   step=0.1,   description='Desired Velocity', layout=layout, style=style),
    'system_noise_std':   widgets.FloatSlider(value=0.0,  min=0.0,    max=0.05, step=0.01,  readout_format='.3f', description='System Noise', layout=layout, style=style),
    'sensor_noise_std':   widgets.FloatSlider(value=0.0,  min=0.0,    max=0.05, step=0.01,  readout_format='.3f', description='Sensor Noise', layout=layout, style=style),
    'time_delta':         widgets.FloatSlider(value=1.0,   min=0.001,  max=5.0,  step=0.01,  description='Time Step', layout=layout, style=style),
    'p_gain':             widgets.FloatSlider(value=0.0,  min=-1.0,   max=3.0,  step=0.01,  description='P Gain', layout=layout, style=style),
    'd_gain':             widgets.FloatSlider(value=0.0,  min=-1.0,   max=3.0,  step=0.01,  description='D Gain', layout=layout, style=style),
    'i_gain':             widgets.FloatSlider(value=0.0,  min=-1.0,   max=3.0,  step=0.01,  description='I Gain', layout=layout, style=style),
    'eps':                widgets.FloatSlider(value=0.001, min=0.0001, max=0.1,  step=0.001, readout_format='.4f',description='eps', layout=layout, style=style),
    'max_steps':          widgets.FloatSlider(value=200,   min=1,      max=1000, step=10,    description='Max Steps', layout=layout, style=style),
    'max_time':           widgets.FloatSlider(value=200,   min=1,      max=1000, step=10,    description='Max Time', layout=layout, style=style)
}

@interact(**widget_sliders)
def run_p_control(init_state, init_velocity, desired_state, desired_velocity, system_noise_std, sensor_noise_std, time_delta, 
                  p_gain, d_gain, i_gain, eps, max_steps, max_time):
    
    system = MassSystem(init_state, init_velocity, system_noise_std, time_delta)
    sensor = Sensor(noise_std=sensor_noise_std)
    
    pid_controller = PidController(p_gain, d_gain, i_gain)

    output_generator = closed_loop(system, pid_controller, sensor, desired_state, init_velocity, 
                                   eps, time_delta, max_steps, max_time, print_debug=PRINT_DEBUG)


    fig = plot_control_loop_output(output_generator, pid_controller)